In [1]:
!pip install ollama

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ollama
import pandas as pd
import json
import re

In [3]:
def clean_string(input_string):
    # Eliminar caracteres especiales, dejando solo letras, números y espacios
    cleaned_string = re.sub(r'[^A-Za-z0-9\s]', '', input_string)
    
    # Eliminar espacios adicionales
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()
    
    return cleaned_string.lower()

In [4]:
def extract_address(content):
    response = ollama.chat(
        model='addr-extractor-codegemma-instruct:0.1',
        messages=[{
            'role': 'user',
            'content': content,
            'stream': True
        }]
    )
    # Devuelve el contenido extraído de la respuesta
    return response['message']['content'].split(",")

In [5]:
extract_address("LA CONCEPCION 191, OFICINA 901,PROVIDENCIA,PROVIDENCIA,PROVIDENCIA")

['Santiago', ' Providencia']

In [6]:
extract_address("Ruta 5 SUR, KM 649 LOTE C LAUTARO,CH,LE,LAUTARO,LAUTARO")

['Santiago', 'Lautaro']

In [7]:
extract_address("Ruta 5 sur km 649 SECTOR LA CANTERA, 4860000 Lautaro, Araucanía")

['Lautaro', ' Araucanía']

In [8]:
extract_address("SAN PEDRO DE QUILLOTA CALLE SANTA R,VALPARAISO,VALPARAISO,VALPARAISO")

['Valparaiso', 'Valparaiso']

# Testeo

In [9]:
# Leer el archivo Excel y verificar direcciones
def check_address_from_excel(file_path):
    df = pd.read_excel(file_path)
    
    total_matches = 0
    #total_checks = len(df)  # Contar hasta un máximo de 10 direcciones
    total_checks = len(df)
    
    for index, row in df.iterrows():
        if index >= total_checks:  # Limitar a 10 iteraciones
            break

        direccion = row['Direccion']
        comuna_esperada = row['Comuna'].lower().strip()
        
        response = extract_address(direccion)
        ciudad_inferida = clean_string(response[0])
        comuna_extraida = clean_string(response[1])
        
        print("\n")
        print(f"Dirección: {direccion}")
        print(f"Respuesta RAW: {response}")
        print("-----------------------------------")
        print(f"Ciudad inferida: {ciudad_inferida}")
        print(f"Comuna extraída: {comuna_extraida}")
        print(f"Comuna esperada: {comuna_esperada}")
        print("-----------------------------------")
        
        if comuna_extraida == comuna_esperada:
            print("Resultado: Match ✅")
            total_matches += 1 
        else:
            print("Resultado: No Match ❌")
        
        print("-----------------------------------")

    
    # Calcular estadísticas finales
    if total_checks > 0:
        match_percentage = (total_matches / total_checks) * 100
        print(f"Total de coincidencias: {total_matches} de {total_checks}")
        print(f"Porcentaje de aciertos: {match_percentage:.2f}%")
    else:
        print("No se realizaron comprobaciones.")

# Llamar a la función pasándole el path de tu archivo Excel
file_path = "direcciones.xlsx"
check_address_from_excel(file_path)



Dirección: PDTE. RIESCO 5685. OFICINA DE PARTES.,LAS CONDES REGION METROPOLITANA,LAS CONDES,LAS CONDES
Respuesta RAW: ['Santiago', 'Las Condes']
-----------------------------------
Ciudad inferida: santiago
Comuna extraída: las condes
Comuna esperada: las condes
-----------------------------------
Resultado: Match ✅
-----------------------------------


Dirección: CARLOS WILSON 1490 PROVIDENCIA,,SANTIAGO CHILE,SANTIAGO,SANTIAGO
Respuesta RAW: ['Santiago', 'Santiago']
-----------------------------------
Ciudad inferida: santiago
Comuna extraída: santiago
Comuna esperada: providencia
-----------------------------------
Resultado: No Match ❌
-----------------------------------


Dirección: PROVIDENCIA AVENUE 1760,OFFICE 1604,SANTIAGO,SANTIAGO
Respuesta RAW: ['Santiago', 'Santiago']
-----------------------------------
Ciudad inferida: santiago
Comuna extraída: santiago
Comuna esperada: providencia
-----------------------------------
Resultado: No Match ❌
---------------------------------